In [1]:
import requests
import time

URL = 'http://localhost:8000'

def start(user_key, problem_id, number_of_elevators):
    uri = URL+'/start/'+user_key+'/'+str(problem_id)+'/'+str(number_of_elevators)
    return requests.post(uri).json()

def oncalls(token):
    uri = URL + '/oncalls'
    return requests.get(uri, headers={'X-Auth-Token':token}).json()

def action(token, commands):
    uri = URL + '/action'
    return requests.post(uri, headers={'X-Auth-Token':token}, json={'commands':commands} ).json()


if __name__ == '__main__':
    problem_num = 1
    elevator_num = 4
    elevator_limit = 8
    top_floor = 25
    bottom_floor = 1
    start_ret = start('lsn1106', problem_num, elevator_num)
    
    token = start_ret['token']
    print('토큰: ',token)
    
    while True:
        oncalls_ret = oncalls(token)
        
        if oncalls_ret['is_end']: break
        
        #oncall로 받은 정보들
        #calls : 콜들
        #elevators : 엘레베이터들
        calls = oncalls_ret['calls']        
        elevators = oncalls_ret['elevators']

        #action을 위한 commands를 각 엘레베이터마다 만듬
        commands = [{} for i in range(elevator_num)]
        
        print("timestamp : ", oncalls_ret['timestamp'])
        
        
        
        cnt=0
        one_stop = False
        for elevator in elevators:
            print("콜 상황")
            for passenger in calls :
                print('승객 id: ', passenger['id'])
                print('출발지: ', passenger['start'])
                print('도착지: ', passenger['end'])
            print('\n')
            
            print('엘레베이터 번호 : ', elevator['id'])
            print('현재 층: ', elevator['floor'])
            print('현재 상태: ', elevator['status'])
            print('엘레베이터 내의 승객수: ', len(elevator['passengers']))
              
            elevator_id = elevator['id']
            floor = elevator['floor']
            passengers = elevator['passengers']
            status = elevator['status']
            
            commands[cnt]['elevator_id'] = elevator_id
            command_str = ""
            call_ids = []
            
            elev_dest = 0 #엘베 내 어떤 승객의 목적지
            call_from = 0 #엘베 밖 어떤 승객의 위치            
                                      
            id_drop = [] #내릴 승객들
            id_board = [] #태울 승객들
            
            for person in passengers:
                if person['end'] == floor : 
                    id_drop.append(person['id'])
                else : 
                    elev_dest = person['end']
                    
            lim = elevator_limit - (len(passengers) - len(id_drop))
                
            for person in calls:
                if person['start'] == floor : 
                    if(len(id_board)<lim):
                        id_board.append(person['id'])
                else : 
                    call_from = person['start']
                
            flag = len(id_drop + id_board) #지금 층에서 내리거나 탈 인원 수
            
                                      
            if status == 'STOPPED':                                
                #지금 내릴/태울 승객있음    
                if flag :
                    command_str = 'OPEN'
                else:
                    #엘베 내 승객 있음
                    if elev_dest :
                        if elev_dest < floor: command_str = 'DOWN'
                        else: command_str = 'UP'
                    #엘베 밖 기다리는 승객 있음
                    elif call_from:
                        if call_from < floor: command_str = 'DOWN'
                        else: command_str = 'UP'                        
                    #아무도 없음    
                    else:
                        command_str = 'STOP'
                                                                           
            elif status == 'UPWARD':
                #내릴 승객있거나 꼭대기층
                if len(id_drop) or floor == top_floor:
                    command_str = 'STOP'
                    one_stop = True
                elif len(id_board) and one_stop==False:
                    command_str = 'STOP'
                    one_stop = True
                else:
                    command_str = 'UP'
                
            elif status == 'DOWNWARD':
                #내릴/태울 승객있거나 1층
                if len(id_drop) or floor == bottom_floor:
                    command_str = 'STOP'
                    one_stop = True
                elif len(id_board) and one_stop==False:
                    command_str = 'STOP'
                    one_stop = True
                else:
                    command_str = 'DOWN'
                
            elif status == 'OPENED':                                    
                #내릴 승객 존재
                if len(id_drop)!=0: 
                    command_str = 'EXIT'
                    call_ids = id_drop
                else :                                                          
                    #다 내리고 태울 승객 존재
                    if len(id_board) != 0: 
                        command_str = 'ENTER'
                        call_ids = id_board
                        
                        for person in calls:
                            if person['id'] in id_board:
                                calls.remove(person)
                    else:
                        command_str = 'CLOSE'
                        
                        
            commands[cnt]['command'] = command_str
            commands[cnt]['call_ids'] = call_ids
            cnt+=1
            #엘레베이터 순회 종료
            print('명령 : '+command_str)
        print('\n\n\n')
        
        action(token, commands)

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

In [10]:
command_str = ["" for i in range(4)]

In [11]:
command_str

['', '', '', '']

In [12]:
command_str = [for i in range(elevator_num)]

SyntaxError: invalid syntax (<ipython-input-12-f68efb2f6b73>, line 1)

In [13]:
commands = [{} for i in range(elevator_num)]

In [14]:
commands

[{}, {}, {}, {}]

In [ ]:
import requests
import time

URL = 'http://localhost:8000'

def start(user_key, problem_id, number_of_elevators):
    uri = URL+'/start/'+user_key+'/'+str(problem_id)+'/'+str(number_of_elevators)
    return requests.post(uri).json()

def oncalls(token):
    uri = URL + '/oncalls'
    return requests.get(uri, headers={'X-Auth-Token':token}).json()

def action(token, commands):
    uri = URL + '/action'
    return requests.post(uri, headers={'X-Auth-Token':token}, json={'commands':commands} ).json()


if __name__ == '__main__':
    problem_num = 1
    elevator_num = 4
    elevator_limit = 8
    top_floor = 25
    bottom_floor = 1
    start_ret = start('lsn1106', problem_num, elevator_num)
    
    token = start_ret['token']
    print('토큰: ',token)
    
    while True:
        oncalls_ret = oncalls(token)
        
        if oncalls_ret['is_end']: break
        
        #oncall로 받은 정보들
        #calls : 콜들
        #elevators : 엘레베이터들
        calls = oncalls_ret['calls']        
        elevators = oncalls_ret['elevators']

        #action을 위한 commands를 각 엘레베이터마다 만듬
        commands = [{} for i in range(elevator_num)]
        
        print("timestamp : ", oncalls_ret['timestamp'])
        
        
        
        cnt=0
        one_stop = False
        for elevator in elevators:
            print("콜 상황")
            for passenger in calls :
                print('승객 id: ', passenger['id'])
                print('출발지: ', passenger['start'])
                print('도착지: ', passenger['end'])
            print('\n')
            
            print('엘레베이터 번호 : ', elevator['id'])
            print('현재 층: ', elevator['floor'])
            print('현재 상태: ', elevator['status'])
            print('엘레베이터 내의 승객수: ', len(elevator['passengers']))
              
            elevator_id = elevator['id']
            floor = elevator['floor']
            passengers = elevator['passengers']
            status = elevator['status']
            
            commands[cnt]['elevator_id'] = elevator_id
            command_str = ""
            call_ids = []
            
            elev_dest = 0 #엘베 내 어떤 승객의 목적지
            call_from = 0 #엘베 밖 어떤 승객의 위치            
                                      
            id_drop = [] #내릴 승객들
            id_board = [] #태울 승객들
            
            for person in passengers:
                if person['end'] == floor : 
                    id_drop.append(person['id'])
                else : 
                    elev_dest = person['end']
                    
            lim = elevator_limit - (len(passengers) - len(id_drop))
                
            for person in calls:
                if person['start'] == floor : 
                    if(len(id_board)<lim):
                        id_board.append(person['id'])
                else : 
                    call_from = person['start']
                
            flag = len(id_drop + id_board) #지금 층에서 내리거나 탈 인원 수
            
                                      
            if status == 'STOPPED':                                
                #지금 내릴/태울 승객있음    
                if flag :
                    command_str = 'OPEN'
                else:
                    #엘베 내 승객 있음
                    if elev_dest :
                        if elev_dest < floor: command_str = 'DOWN'
                        else: command_str = 'UP'
                    #엘베 밖 기다리는 승객 있음
                    elif call_from:
                        if call_from < floor: command_str = 'DOWN'
                        else: command_str = 'UP'                        
                    #아무도 없음    
                    else:
                        command_str = 'STOP'
                                                                           
            elif status == 'UPWARD':
                #내릴 승객있거나 꼭대기층
                if len(id_drop) or floor == top_floor:
                    command_str = 'STOP'
                    one_stop = True
                elif len(id_board) and one_stop==False:
                    command_str = 'STOP'
                    one_stop = True
                else:
                    command_str = 'UP'
                
            elif status == 'DOWNWARD':
                #내릴/태울 승객있거나 1층
                if len(id_drop) or floor == bottom_floor:
                    command_str = 'STOP'
                    one_stop = True
                elif len(id_board) and one_stop==False:
                    command_str = 'STOP'
                    one_stop = True
                else:
                    command_str = 'DOWN'
                
            elif status == 'OPENED':                                    
                #내릴 승객 존재
                if len(id_drop)!=0: 
                    command_str = 'EXIT'
                    call_ids = id_drop
                else :                                                          
                    #다 내리고 태울 승객 존재
                    if len(id_board) != 0: 
                        command_str = 'ENTER'
                        call_ids = id_board
                        
                        for person in calls:
                            if person['id'] in id_board:
                                calls.remove(person)
                    else:
                        command_str = 'CLOSE'
                        
                        
            commands[cnt]['command'] = command_str
            commands[cnt]['call_ids'] = call_ids
            cnt+=1
            #엘레베이터 순회 종료
            print('명령 : '+command_str)
        print('\n\n\n')
        
        action(token, commands)